In [3]:
# !pip install rdflib
# !pip install morph-kgc
# !pip install owlrl
# !pip install sqlalchemy

In [4]:
import morph_kgc
g = morph_kgc.materialize('../mappings/config.ini')

g.serialize(destination='resultado.nt', format='nt')
print("Archivo resultado.nt generado")

DEBUG | 2025-11-16 14:49:34,104 | CONFIGURATION: {'output_file': '../rdf/tripletas-Final', 'number_of_processes': '1', 'logging_level': 'DEBUG', 'safe_percent_encoding': ":/?#[]@!$&'()*+,;=%-._~", 'na_values': ',nan', 'literal_escaping_chars': '",\n,\r', 'read_parsed_mappings_path': '', 'write_parsed_mappings_path': '', 'mapping_partitioning': 'PARTIAL-AGGREGATIONS', 'logging_file': '', 'udfs': '', 'output_kafka_server': '', 'output_kafka_topic': '', 'output_dir': '', 'output_format': 'N-TRIPLES', 'only_printable_chars': 'no', 'infer_sql_datatypes': 'no'}
DEBUG | 2025-11-16 14:49:34,106 | DATA SOURCE `DataSourceCSV`: {'mappings': 'rml-map-with-links-Final.ttl'}
INFO | 2025-11-16 14:49:34,107 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.


FileNotFoundError: [Errno 2] No such file or directory: 'rml-map-with-links-Final.ttl'

In [ ]:
from rdflib import Graph
from owlrl import DeductiveClosure, RDFS_Semantics

g = Graph()

g.parse("../ontology/ontology.ttl", format="turtle")
g.parse("resultado.nt", format="nt")

# DeductiveClosure(RDFS_Semantics).expand(g)

<Graph identifier=Nca307fa3050c4389a459473fd15bad2e (<class 'rdflib.graph.Graph'>)>

In [ ]:
PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#> 
PREFIX schema: <https://schema.org/> 
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#> 
PREFIX dbo:    <http://dbpedia.org/ontology/> 
PREFIX owl:    <http://www.w3.org/2002/07/owl#> 
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#> 
"""


print(f"Triples cargados: {len(g)}")

# Mostrar las primeras 10 tripletas
for i, (s, p, o) in enumerate(g):
    print(s, p, o)
    if i >= 55:
        break

Triples cargados: 8221
http://smartcity.linkeddata.es/lcc/resource/facility/EducationalInstitution/Universidad-Complutense-de-Madrid.-Facultad-de-Filologia https://schema.org/postalCode 28040
http://smartcity.linkeddata.es/lcc/resource/facility/EducationalInstitution/Colegio-Universitario-de-Estudios-Financieros https://schema.org/name Colegio Universitario de Estudios Financieros
http://smartcity.linkeddata.es/lcc/resource/facility/SportsFacility/Centro-Deportivo-Municipal-La-Cebada-%28antigua-La-Latina%29 http://www.w3.org/2003/01/geo/wgs84_pos#lat 40.410546084176566
http://smartcity.linkeddata.es/lcc/resource/facility/Park/Parque-de-la-Amistad http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://smartcity.linkeddata.es/lcc/ontology#Facility
http://smartcity.linkeddata.es/lcc/resource/facility/Park/Parque-Eva-Duarte http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://smartcity.linkeddata.es/lcc/ontology#Facility
http://smartcity.linkeddata.es/lcc/resource/facility/SportsFacility

In [ ]:
print(f"Triples cargados: {len(g)}")

PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
"""

def qrun(q, limit=50):
    q = PREFIXES + q
    res = g.query(q)
    print(f"--- {len(res)} filas ---")
    for i,row in enumerate(res):
        print([str(x) if x is not None else None for x in row])
        if i >= limit-1: break

Triples cargados: 8221


In [ ]:
########################################################################################
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter
########################################################################################

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:Facility .
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type
""")


# --- 8 filas ---
# ['Educational Institution', '117']
# ['Faculty', '35']
# ['Library', '50']
# ['Other Centers', '13']
# ['Park or Garden', '203']
# ['Residence Hall', '48']
# ['Sport Facility', '81']
# ['University School', '21']

--- 4 filas ---
['Educational Institution', '117']
['Library', '50']
['Park or Garden', '203']
['Sport Facility', '81']


In [ ]:
qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:EducationalInstitution .
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type
""")


--- 4 filas ---
['Faculty', '35']
['Other Centers', '13']
['Residence Hall', '48']
['University School', '21']


In [ ]:
########################################################################################
# ✅ Número de individuos de cada subclase DIRECTA de Facility, no aparecen subclases de subclases 
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter
########################################################################################

for subtype,label in [
    ("sc:EducationalInstitution","EducationalInstitutions"),
    ("sc:Library","PublicLibraries"),
    ("sc:Park","ParksGardens"),
    ("sc:SportsFacility","SportsCenters")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {subtype} .

  # Filtrar solo subclases directas de Facility
  ?subclass rdfs:subClassOf sc:Facility .
  ?x a ?subclass .

  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### EducationalInstitutions
--- 5 filas ---
['Escuela Técnica Superior de Ingenieros de Caminos, Canales y Puertos. Edificio Retiro']
['Escuela Universitaria de Enfermería. Fundación Jiménez Díaz']
['Universidad Complutense de Madrid. Facultad de Informática']
['Centro de Enseñanza Superior en Humanidades y Ciencias de la Educación Don Bosco']
['Residencia Universitaria Tagaste']

### PublicLibraries
--- 5 filas ---
['Biblioteca Pública Municipal Gabriel García Márquez (Usera)']
['Biblioteca Pública Municipal Iván de Vargas (Centro)']
['Biblioteca Pública Municipal Ángel González (Latina)']
['Biblioteca Pública Municipal Ciudad Lineal (Ciudad Lineal)']
['Centro Cultural Paco Rabal. Biblioteca Pública (Puente de Vallecas)']

### ParksGardens
--- 5 filas ---
['Parque de Las Cruces']
['Parque Eugenia de Montijo']
['Parque Mujeres de Orcasitas']
['Parque San José de Calasanz']
['Jardines de Sabatini']

### SportsCenters
--- 5 filas ---
['Centro Deportivo Municipal Hortaleza']
['Centro Dep

In [ ]:
########################################################################################
# ✅ Tipos de Public Transport, suclases de public transport: (subwary, train, bus)
########################################################################################

qrun("""
SELECT ?type WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:PublicTransport .
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type
""")

--- 3 filas ---
['Bus']
['Subway']
['Train']


In [ ]:
########################################################################################
# ✅ 5 ejemplos de Estaciones de Metro y Tren 
########################################################################################

for ttype, label in [
    ("sc:Subway", "Metro Stations"),  # Cambiamos de SubwayStation a Subway
    ("sc:Train", "Train Stations")  # Cambiamos de TrainStation a Train
]:
    print(f"\n### {label}")
    # Modificamos la consulta para buscar el recurso de transporte (?t)
    qrun(f"""
SELECT DISTINCT ?stationName WHERE {{
  ?t a {ttype} .        
  ?t sc:hasStations ?stationName .
}}
LIMIT 5
""", limit=5)



### Metro Stations
--- 5 filas ---
['Canillejas, Torre Arias']
['Pacífico']
['Pío XII']
['Carabanchel , Eugenia de Montijo']
['Villaverde Bajo Cruce']

### Train Stations
--- 5 filas ---
['Orcasitas']
['El Pozo']
[': Villaverde Bajo']
['Embajadores']
['Príncipe Pío']


In [ ]:
########################################################################################
# ✅ 5 ejemplos de Lineas de Metro, Tren y bus
########################################################################################

for ttype, label in [
    ("sc:Subway", "Metro lines"),  
    ("sc:Train", "Train lines"),  
    ("sc:Bus", "Bus lines") 
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT DISTINCT ?busLine WHERE {{
  ?t a {ttype} .        
  ?t sc:hasLines ?busLine .
}}
LIMIT 5
""", limit=5)


### Metro lines
--- 5 filas ---
['5']
['3']
['9']
['2']
['6']

### Train lines
--- 5 filas ---
['C-5']
['C-2\r\nC-7\r\nC-8']
['C-3\r\nC-4']
['C-1\r\nC-7\r\nC-10']
['C-3\r\nC-7\r\nC-8']

### Bus lines
--- 5 filas ---
['104\r\n77\r\n140\r\n153']
['18\r\n62']
['34']
['10\r\n14\r\n32\r\n56\r\n63\r\n143\r\n145']
['3\r\n60\r\n148']


In [ ]:
########################################################################################
# ✅ Número de Municipios, Distritos y Barrios distintos
########################################################################################

qrun("""
SELECT ?type (COUNT(DISTINCT ?entity) AS ?count)
WHERE {
  { ?entity a dbo:Municipality  BIND("Municipality"  AS ?type) } UNION
  { ?entity a dbo:District      BIND("District"      AS ?type) } UNION
  { ?entity a dbo:Neighbourhood BIND("Neighbourhood" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
     
""")

--- 3 filas ---
['District', '22']
['Municipality', '1']
['Neighbourhood', '118']


In [ ]:
########################################################################################
# ✅  5 ejemplos de Municipio, Distrito y Barrio (solo nombre)
########################################################################################

for ctype,label in [
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood"),
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {ctype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Municipality
--- 1 filas ---
['MADRID']

### District
--- 5 filas ---
['TETUAN']
['CHAMARTIN']
['DISTRITO']
['FUENCARRAL-EL PARDO']
['CENTRO']

### Neighbourhood
--- 5 filas ---
['CASTILLA']
['JUSTICIA']
['ALMENDRALES']
['MIRASIERRA']
['PROSPERIDAD']


In [ ]:
########################################################################################
# ✅  Comprobación jerarquía Municipio→Distrito→Barrio
########################################################################################

print("### Neigbourhood → District")
qrun("""
SELECT ?municipality ?district WHERE {
  ?m a dbo:Neighbourhood ; schema:name ?municipality ; sc:locatedInDistrict ?d .
  OPTIONAL { ?d schema:name ?district }
}
ORDER BY ?municipality ?district
""", limit=5)

print("\n### District → Municipality")
qrun("""
SELECT ?district ?neighbourhood WHERE {
  ?d a dbo:District ; schema:name ?district ; sc:locatedInMunicipality ?n .
  OPTIONAL { ?n schema:name ?neighbourhood }
}
ORDER BY ?district ?neighbourhood
""", limit=5)


### Neigbourhood → District
--- 119 filas ---
['ABRANTES', 'CARABANCHEL']
['ACACIAS', 'ARGANZUELA']
['ADELFAS', 'RETIRO']
['ALAMEDA DE OSUNA', 'BARAJAS']
['ALMAGRO', 'CHAMBERI']

### District → Municipality
--- 22 filas ---
['ARGANZUELA', 'MADRID']
['BARAJAS', 'MADRID']
['CARABANCHEL', 'MADRID']
['CENTRO', 'MADRID']
['CHAMARTIN', 'MADRID']


In [ ]:
################################################################################################################
# ✅  Comprobar que distintas facilities, parques, sportscenter ... estan asociadas a su → Barrio
################################################################################################################

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("sc:Library","PublicLibraries"),
    ("sc:Park","ParksGardens"),
    ("sc:SportsFacility","SportsCenters")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
BIND(COALESCE(?fraw, REPLACE(REPLACE(STR(?f), '^.*/', ''), '%20', ' ')) AS ?facility)
BIND(COALESCE(?nraw, REPLACE(REPLACE(STR(?n), '^.*/', ''), '%20', ' ')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)



### Universities → Neighbourhood
--- 5 filas ---
['Escuela Técnica Superior de Ingenieros de Caminos, Canales y Puertos. Edificio Retiro', 'LOS JERONIMOS']
['Escuela Universitaria de Enfermería. Fundación Jiménez Díaz', 'ARGUELLES']
['Universidad Complutense de Madrid. Facultad de Informática', 'CIUDAD UNIVERSITARIA']
['Centro de Enseñanza Superior en Humanidades y Ciencias de la Educación Don Bosco', 'CIUDAD UNIVERSITARIA']
['Residencia Universitaria Tagaste', 'GOYA']

### PublicLibraries → Neighbourhood
--- 5 filas ---
['Biblioteca Pública Municipal Gabriel García Márquez (Usera)', 'ORCASUR']
['Biblioteca Pública Municipal Iván de Vargas (Centro)', 'PALACIO']
['Biblioteca Pública Municipal Ángel González (Latina)', 'ALUCHE']
['Biblioteca Pública Municipal Ciudad Lineal (Ciudad Lineal)', 'PUEBLO NUEVO']
['Centro Cultural Paco Rabal. Biblioteca Pública (Puente de Vallecas)', 'PALOMERAS BAJAS']

### ParksGardens → Neighbourhood
--- 5 filas ---
['Parque de Las Cruces', 'BUENAVISTA']
['P

In [ ]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:Facility","Facilities")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
BIND(COALESCE(?fraw, REPLACE(REPLACE(STR(?f), '^.*/', ''), '%20', ' ')) AS ?facility)
BIND(COALESCE(?nraw, REPLACE(REPLACE(STR(?n), '^.*/', ''), '%20', ' ')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)


### Facilities → Neighbourhood
--- 5 filas ---
['Parque de la Amistad', 'LOS ROSALES']
['Parque Eva Duarte', 'GUINDALERA']
['Jardines de la Plaza de Chamberí', 'ALMAGRO']
['Parque Sancho Dávila', 'FUENTE DEL BERRO']
['Jardín del Maestro Padilla', 'IMPERIAL']
